In [1]:
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd

#mylist = ['PVRINOX','COLPAL','IDEA','ABCAPITAL','APOLLOTYRE','BANDHANBNK']
mylist1 = ['ABCAPITAL.NS', 'APOLLOTYRE.NS', 'ABFRL.NS', 'ADANIPORTS.NS', 'APOLLOHOSP.NS', 
           'AUROPHARMA.NS', 'AXISBANK.NS', 'ASHOKLEY.NS', 'AMBUJACEM.NS', 'BATAINDIA.NS', 
           'BERGEPAINT.NS', 'BHEL.NS', 'BRITANNIA.NS', 'COALINDIA.NS', 'COLPAL.NS', 
           'ASIANPAINT.NS', 'DABUR.NS', 'EICHERMOT.NS', 'GAIL.NS', 'GODREJPROP.NS', 
           'BAJAJ-AUTO.NS', 'GUJGASLTD.NS', 'HAVELLS.NS', 'HCLTECH.NS', 'BPCL.NS', 
           'HDFCBANK.NS', 'HDFCLIFE.NS', 'HINDALCO.NS', 'HINDCOPPER.NS', 'HINDUNILVR.NS', 
           'ICICIBANK.NS', 'ICICIGI.NS', 'ICICIPRULI.NS', 'IDFC.NS', 'IDFCFIRSTB.NS', 
           'INDIGO.NS', 'INFY.NS', 'IOC.NS', 'IRCTC.NS', 'JINDALSTEL.NS', 'JKCEMENT.NS', 
           'JUBLFOOD.NS', 'KOTAKBANK.NS', 'LT.NS', 'LTIM.NS', 'LTTS.NS', 'MARUTI.NS', 
           'MCDOWELL-N.NS', 'MFSL.NS', 'MRF.NS', 'NATIONALUM.NS', 'BANKBARODA.NS', 'NESTLEIND.NS', 
           'NMDC.NS', 'ONGC.NS', 'PIDILITIND.NS', 'PVRINOX.NS', 'RELIANCE.NS', 'SBICARD.NS', 
           'SBILIFE.NS', 'SBIN.NS', 'SHREECEM.NS', 'TATACHEM.NS', 'CIPLA.NS', 'TATACOMM.NS', 
           'TATACONSUM.NS', 'HAL.NS', 'TATAMOTORS.NS', 'TATAPOWER.NS', 'TCS.NS', 'TITAN.NS', 
           'BHARTIARTL.NS', 'TVSMOTOR.NS', 'UBL.NS', 'ULTRACEMCO.NS', 'BIOCON.NS', 'WIPRO.NS', 
           'BOSCHLTD.NS', 'ZEEL.NS', 'HDFCAMC.NS', 'HEROMOTOCO.NS', 'HINDPETRO.NS', 'L&TFH.NS', 
           'M&M.NS', 'POWERGRID.NS', 'NTPC.NS', 'PNB.NS', 'VEDL.NS', 'SAIL.NS', 'VOLTAS.NS', 
           'CANBK.NS', 'IGL.NS', 'ITC.NS', 'JSWSTEEL.NS', 'TATASTEEL.NS', 'SIEMENS.NS', 
           'CONCOR.NS', 'IDEA.NS', 'TECHM.NS']
current_date = datetime.today().date().strftime("%Y-%m-%d")


start_date = '2022-06-01' #input("Enter the start date (YYYY-MM-DD): ")
#end_date   = '2023-10-01'  #input("Enter the end date (YYYY-MM-DD): ")
end_date   = datetime.today().date().strftime("%Y-%m-%d")

initial_money = 5000

# Download the stock price data from Yahoo Finance API
data = yf.download(mylist1, start=start_date, end=end_date)
df = pd.DataFrame(data) 
df1 = df.stack().reset_index().rename(index=str, columns={"level_1": "Symbol"}).sort_values(['Symbol','Date']).reset_index(drop=True)
df1

In [4]:
import numpy as np

def count_buy_sell(row):
    return (row == 'Buy').sum() + (row == 'Sell').sum()


def process_one(data):
    # Calculate the short-term and long-term Simple Moving Averages (SMAs)
    sma_short = data['Close'].rolling(window=20).mean()
    sma_long = data['Close'].rolling(window=50).mean()

    # Determine the Buy or Sell signals based on the SMA crossovers
    signals = [None]*len(sma_short)
    for i in range(1, len(sma_short)):
        if sma_short[i] > sma_long[i] and sma_short[i-1] <= sma_long[i-1]:
            signals[i] = 'Buy'
            #print(f"Buy signal detected on {data.index[i].date()}")
        elif sma_short[i] < sma_long[i] and sma_short[i-1] >= sma_long[i-1]:
            signals[i] = 'Sell'
            #print(f"Sell signal detected on {data.index[i].date()}")

    # Add the signals to the data frame
    data['Signal'] = signals
    
    # Initialize variables for tracking the buying and selling prices, as well as the number of shares and cash balance
    buy_price = 0
    sell_price = 0
    num_shares = 0
    cash_balance = initial_money

    # Simulate buying and selling based on the generated signals
    for i in range(len(data)):
        if data['Signal'][i] == 'Buy':
            if cash_balance > 0:
                num_shares = cash_balance / data['Close'][i]
                buy_price = data['Close'][i]
                cash_balance = 0
        elif data['Signal'][i] == 'Sell':
            if num_shares > 0:
                sell_price = data['Close'][i]
                cash_balance = num_shares * sell_price
                num_shares = 0

    # Calculate the final balance based on the last stock price and remaining number of shares
    if num_shares > 0:
        final_balance = num_shares * data['Close'][-1]
    else:
        final_balance = cash_balance

    # Print the final balance and profit/loss
    #print(f"Final balance: Rs.{final_balance:.2f}")
    #print(f"Profit/Loss: Rs.{final_balance - initial_money:.2f}")
    return data
    

dfc = pd.DataFrame()
for item in mylist1:
    data1 = df1[df1['Symbol']==item].reset_index(drop=True).set_index('Date')
    dfc[item] = process_one(data1)['Signal']

dfc = dfc.tail(7).T
dfc = dfc.fillna('')

dfc['count'] = dfc.apply(count_buy_sell, axis=1)



#dfc= dfc.sort_values(list(dfc.columns),ascending=False)
#dfc.to_excel("stocks.xlsx")
dfc=dfc[dfc['count']>0]
dfc.T

,APOLLOHOSP.NS,ASHOKLEY.NS,BAJAJ-AUTO.NS,HDFCLIFE.NS,ICICIBANK.NS,ICICIPRULI.NS,NESTLEIND.NS,SBIN.NS,TATAMOTORS.NS,L&TFH.NS,IGL.NS,JSWSTEEL.NS
Date,,,,,,,,,,,,
2023-09-21 00:00:00,,,Buy,,,,,,,,Buy,
2023-09-22 00:00:00,,,,Buy,Sell,,,,,,,
2023-09-25 00:00:00,,,,,,Buy,,,,,,
2023-09-26 00:00:00,,Sell,,,,,,,,,,Buy
2023-09-27 00:00:00,,,,,,,,,Buy,Buy,,
2023-09-28 00:00:00,,,,,,,,Buy,,,,
2023-09-29 00:00:00,Buy,,,,,,Buy,,,,,
count,1,1,1,1,1,1,1,1,1,1,1,1
